In [1]:
import pandas as pd
import numpy as np
import ast
import csv
import json
import re

In [3]:
def load_json_multiple(segments):
    #chunk = ""
    for segment in segments:
        #chunk += segment
        try:
            yield json.loads(segment)
            #chunk = ""
        except ValueError:
            pass


tweets_json = []

with open('merged_25.json') as f:
    for parsed_json in load_json_multiple(f):
        tweets_json.append(parsed_json)

In [4]:
text = []
for line in tweets_json:
    if line == 420:
        continue
    if 'text' in line:
        text.append(line['text'])

In [5]:
def isEnglish(s):
    try:
        s.encode('ascii')
    except UnicodeEncodeError:
        return False
    else:
        return True

In [6]:
Eng = []
for line in text:
    if isEnglish(line):
        Eng.append(line)

In [7]:
removeMention = []
regexM = r'@\S*(?=\s)'
regexH = r'https://t.co/[a-zA-Z0-9]{10}'
for line in Eng:
    tempM = ""
    tempH = ""
    
    if "@" in line and "https" in line:
        lit = re.findall(regexM, line)
        htp = re.findall(regexH, line)
        for word in lit:
            tempM = line.replace(word, "")
        for word in htp:
            tempH = tempM.replace(word, "")
        
        tp = tempH.replace("#", "")
        removeMention.append(tp)
    
    elif "@" in line:
        lit = re.findall(regexM, line)
        for word in lit:
            tempM = line.replace(word, "")
            
        tp = tempM.replace("#", "")
        removeMention.append(tp)

    elif "https" in line:
        htp = re.findall(regexH, line)
        for word in htp:
            tempH = line.replace(word, "")
        
        tp = tempH.replace("#", "")
        removeMention.append(tp)
        
    else:
        removeMention.append(line)

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

In [9]:
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(removeMention)

In [26]:
true_k = 10
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)
model.fit(X)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=100,
    n_clusters=10, n_init=1, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [27]:
#print("Top terms per cluster:")
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()
# for i in range(true_k):
#     print ("Cluster %d:" % i,)
#     for ind in order_centroids[i, :10]:
#         print (' %s' % terms[ind]),
#     print

In [28]:
w, h = 8, 10;
Matrix = [[0 for x in range(w)] for y in range(h)] 
#print(Matrix)

In [29]:
for i in range(0, true_k-1):
    j = 0
    for ind in order_centroids[i, :8]:
        #print(i, j)
        Matrix[i][j] = ('%s' % terms[ind])
        j += 1

In [30]:
with open('cluster_25.csv', 'w', newline='') as f:
    
    writer = csv.writer(f)
    writer.writerows(Matrix)    

In [32]:
MartixOpmzd = []

In [33]:
#manually changed the cluster, erase the nonsense words.
f = open("cluster_25.csv", 'r')
try:
    reader = csv.reader(f)
    for row in reader:
        MartixOpmzd.append(row)
finally:
    f.close()

In [36]:
MatrixContent = [[] for _ in range(8)]
Unmatched = []
manualLabel = []
EachPoint = [0] * 9

In [37]:
for line in removeMention:
    Point = [0] * 8
    #print(Point)
    temp = line.split(" ")
    for word in temp:
        #print(word)
        for ir in range(0, 7):
            wd = word.lower().strip()
            if wd in MartixOpmzd[ir]:
                Point[ir] += 1
    mmax = Point.index(max(Point))
    if Point.count(Point[mmax]) > 1:
        manualLabel.append(line)
    #print(mmax)
    else:
        if sum(Point) == 0:
            Unmatched.append(line)
            EachPoint[20] += 1
        else:
            if line not in MatrixContent[mmax]:
                MatrixContent[mmax].append(line)
            EachPoint[mmax] += 1

In [13]:
#MatrixContent[1]

In [38]:
EachPoint

[15, 73, 64, 34, 59, 234, 34, 0, 0]

In [39]:
len(Unmatched)

0